In [6]:
import cv2
import numpy as np
import mediapipe as mp
from playsound import playsound
import threading
from collections import deque


In [7]:
RIGHT_EYE = [33, 160, 158, 133, 153, 144, 155]
LEFT_EYE  = [362, 385, 387, 263, 373, 380, 374]

def eye_aspect_ratio(landmarks, eye_indices):
    p1 = np.array([landmarks[eye_indices[0]].x, landmarks[eye_indices[0]].y])
    p2 = np.array([landmarks[eye_indices[1]].x, landmarks[eye_indices[1]].y])
    p3 = np.array([landmarks[eye_indices[2]].x, landmarks[eye_indices[2]].y])
    p4 = np.array([landmarks[eye_indices[3]].x, landmarks[eye_indices[3]].y])
    p5 = np.array([landmarks[eye_indices[4]].x, landmarks[eye_indices[4]].y])
    p6 = np.array([landmarks[eye_indices[5]].x, landmarks[eye_indices[5]].y])

    vertical1 = np.linalg.norm(p2 - p6)
    vertical2 = np.linalg.norm(p3 - p5)
    horizontal = np.linalg.norm(p1 - p4)

    ear = (vertical1 + vertical2) / (2.0 * horizontal)
    return ear

In [8]:
def sound_alarm():
    playsound("alarm.wav")

def start_alarm():
    t = threading.Thread(target=sound_alarm, daemon=True)
    t.start()


In [9]:
EAR_THRESH = 0.25
CONSEC_FRAMES = 20
counter = 0
alarm_on = False
buffer = deque(maxlen=CONSEC_FRAMES)

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        landmarks = results.multi_face_landmarks[0].landmark
        right_ear = eye_aspect_ratio(landmarks, RIGHT_EYE)
        left_ear = eye_aspect_ratio(landmarks, LEFT_EYE)
        avg_ear = (right_ear + left_ear) / 2.0

        cv2.putText(frame, f"EAR: {avg_ear:.2f}", (30, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                    (0, 255, 0) if avg_ear > EAR_THRESH else (0, 0, 255), 2)

        if avg_ear < EAR_THRESH:
            buffer.append(1)
        else:
            buffer.append(0)

        if sum(buffer) == CONSEC_FRAMES and not alarm_on:
            alarm_on = True
            start_alarm()

        if alarm_on and sum(buffer) < CONSEC_FRAMES // 2:
            alarm_on = False

        if alarm_on:
            cv2.putText(frame, "DROWSINESS DETECTED!", (150, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

    cv2.imshow("Driver Drowsiness Detection", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC to stop
        break

cap.release()
cv2.destroyAllWindows()
